In [16]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import requests
from bs4 import BeautifulSoup
import time 
from tqdm import tqdm
import random
import pandas as pd

In [17]:
def get_sitemap_urls(sitemap_url):
    response = requests.get(sitemap_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        urls = [loc.text for loc in soup.find_all('loc')]
        return urls
    else:
        print(f"Failed to fetch sitemap. Status Code: {response.status_code}")
        return []

In [18]:
def unique(Orignal_list):
    unique_list = [item for index, item in enumerate(Orignal_list) if item not in Orignal_list[:index]]
    return unique_list

In [19]:
allowed_user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/14.1.2",
    "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"   
]
Product_details=[]

In [5]:
sitemap_url = 'https://www.edenbrothers.com/sitemap_products_1.xml?from=7678123901180&to=8179030458620'
sitemap_urls = get_sitemap_urls(sitemap_url)
Pro_URL= []
for i in sitemap_urls:
    if 'https://www.edenbrothers.com/products' in i:
        Pro_URL.append(i)
df = pd.DataFrame(Pro_URL)
df.to_csv('Links.csv',index=False)

C:\Users\Rahul Manocha\anaconda3\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [44]:
Pro_URL.index('https://www.edenbrothers.com/products/sunset-flower-seed-mix')
Pro_URL[2044]

'https://www.edenbrothers.com/products/power-couple-flower-seed-mix'

In [47]:
def get_product_info(URL):
    driver.get(URL)
    time.sleep(1)
    df = [] 
    wait = WebDriverWait(driver, 3)
    try:
        try: 
            FSeason = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/main/section[1]/section/div[2]/div[2]/div/div[3]/div[contains(@class, 'force-active-season')]")))
            Season = FSeason.text
        except Exception as e:
            SSeason = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/main/section[1]/section/div[2]/div[2]/div/div[2]/div[contains(@class, 'force-active-season')]")))
            Season = SSeason.text
            
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        P_name_element = driver.find_element(By.XPATH, '/html/body/main/section[1]/section/div[2]/div[2]/div/div[1]/h1')
        Rating_element = soup.find('span', class_="stamped-summary-text-1")
        ReviewCount_element = soup.find('span', class_="stamped-summary-text")
        ProductName = P_name_element.text.strip()
        ReviewCount = ReviewCount_element.text.strip() if ReviewCount_element else 'Based on 0 Reviews'
        Rating = Rating_element.text.strip() if Rating_element else '0'
        variant = soup.find_all('tr')
        sub_pro = [span.text for row in variant for span in row.find_all('span')[:1] if row.find_all('span')]
        prices = []
        for row in variant:
            td_tags = row.find_all('td')
            for td in td_tags:
                if 'class' not in td.attrs and td.text.strip().startswith('$'):
                    prices.append(td.text.strip())
            cleaned_prices = [price.split('\n')[0].strip() if '\n' in price else price for price in prices]
        RegularPrice_element = soup.find_all('span', class_='original-price')
        RegularPrice= [RegularPrice_element[i].text.replace('Original Price:', '').strip() for i in range(len(RegularPrice_element))]
        SalePrice_element = soup.find_all('span', class_='sale-price')
        SalePrice= [SalePrice_element[i].text.replace('Sale Price:', '').strip() for i in range(len(SalePrice_element))]
        sub_pro = unique(sub_pro)
        if RegularPrice:
            for i in range(len(sub_pro)):
                row = {
                        'PageURL': URL,
                        'ProductName': ProductName,
                        'Season': Season,
                        'SubProduct' : sub_pro[i],
                        'RegularPrice': RegularPrice[i],
                        'SalePrice':SalePrice[i],
                        'ReviewCount': ReviewCount,
                        'OverAllRating': Rating
                    }
                df.append(row)
        else:
            for i in range(len(sub_pro)):
                row = {
                        'PageURL': URL,
                        'ProductName': ProductName,
                        'Season': Season,
                        'SubProduct' : sub_pro[i],
                        'RegularPrice': None,
                        'SalePrice':cleaned_prices[i],
                        'ReviewCount': ReviewCount,
                        'OverAllRating': Rating
                    }
                df.append(row)
    except TimeoutException as e:
        paths = ['/html/body/main/section[1]/section/div[2]/div[2]/div/div[2]/input[1]',
                 '/html/body/main/section[1]/section/div[2]/div[2]/div/div[3]/input[1]'
        ]
        for path in paths:
            try:
                Fall_Season = wait.until(EC.element_to_be_clickable((By.XPATH, path)))  
                #Fall_Season.click()
                driver.execute_script("arguments[0].click();", Fall_Season)
                break
            except Exception as e:
                pass
        
        paths = ['/html/body/main/section[1]/section/div[2]/div[2]/div/div[2]/label[1]',
                 '/html/body/main/section[1]/section/div[2]/div[2]/div/div[3]/label[1]'
        ]
        for path in paths:
            try:
                Season = driver.find_element(By.XPATH, path).text
                break
            except Exception as e:
                pass
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        P_name_element = driver.find_element(By.XPATH, '/html/body/main/section[1]/section/div[2]/div[2]/div/div[1]/h1')
        Rating_element = soup.find('span', class_="stamped-summary-text-1")
        ReviewCount_element = soup.find('span', class_="stamped-summary-text")
        ProductName = P_name_element.text.strip()
        ReviewCount = ReviewCount_element.text.strip() if ReviewCount_element else 'Based on 0 Reviews'
        Rating = Rating_element.text.strip() if Rating_element else '0'
        table = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//tbody")))
        rows = table.find_elements(By.XPATH, "//tr[@class='season active-season']")
        subproducts = []
        RegularPrices = []
        SalePrices = []
        for row in rows:
            subproduct_element = row.find_element(By.XPATH, ".//td/span[1]")
            subproduct_text = subproduct_element.text.strip()
            subproducts.append(subproduct_text)
            try: 
                RegularPrice_element = row.find_element(By.XPATH,".//td/span[contains(@class, 'original-price')]")
                RegularPrice = RegularPrice_element.text.strip() if RegularPrice_element else None
                RegularPrices.append(RegularPrice)
            except Exception as e:
                RegularPrice = None
                RegularPrices.append(RegularPrice)
            try:
                SalePrice_element = row.find_element(By.XPATH,".//td/span[contains(@class, 'sale-price')]")
                SalePrice = SalePrice_element.text.strip()
                SalePrices.append(SalePrice)
            except Exception as e:
                SalePrice_element = row.find_element(By.XPATH,".//td[2]")
                SalePrice = SalePrice_element.text.strip()
                SalePrice = SalePrice.split('\n')[0].strip()
                SalePrices.append(SalePrice)
        for i in range(len(subproducts)):
                row = {
                        'PageURL': URL,
                        'ProductName': ProductName,
                        'Season' : Season,
                        'SubProduct' : subproducts[i],
                        'RegularPrice': RegularPrices[i],
                        'SalePrice':SalePrices[i],
                        'ReviewCount': ReviewCount,
                        'OverAllRating': Rating
                    }
                df.append(row)
        paths = [
        '/html/body/main/section[1]/section/div[2]/div[2]/div/div[2]/input[2]',
        '/html/body/main/section[1]/section/div[2]/div[2]/div/div[3]/input[2]'
        ]
        for path in paths:
            try:
                Spring_Season = wait.until(EC.element_to_be_clickable((By.XPATH, path)))  
                #Spring_Season.click()
                driver.execute_script("arguments[0].click();", Spring_Season)
                break
            except Exception as e:
                pass
                
        
        paths = [
        '/html/body/main/section[1]/section/div[2]/div[2]/div/div[2]/label[2]',
        '/html/body/main/section[1]/section/div[2]/div[2]/div/div[3]/label[2]'
        ]
        for path in paths:
            try:
                Season = driver.find_element(By.XPATH,path).text
                break
            except Exception as e:
                pass
                
        
        table = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//tbody")))
        rows = table.find_elements(By.XPATH, "//tr[@class='season inactive-season']")
        subproducts = []
        SalePrices = []
        for row in rows:
            subproduct_element = row.find_element(By.XPATH, ".//td/span[1]")
            subproduct_text = subproduct_element.text.strip()
            price_element = row.find_element(By.XPATH, ".//td[2]")
            SalePrice = price_element.text.strip()
            SalePrice = SalePrice.split('\n')[0].strip()
            subproducts.append(subproduct_text)
            SalePrices.append(SalePrice)
        
        for i in range(len(subproducts)):
                row = {
                        'PageURL': URL,
                        'ProductName': ProductName,
                        'Season' : Season,
                        'SubProduct' : subproducts[i],
                        'RegularPrice': None,
                        'SalePrice':SalePrices[i],
                        'ReviewCount': ReviewCount,
                        'OverAllRating': Rating
                    }
                df.append(row)
    if not df:
        Season = None
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        P_name_element = driver.find_element(By.XPATH, '/html/body/main/section[1]/section/div[2]/div[2]/div/div[1]/h1')
        Rating_element = soup.find('span', class_="stamped-summary-text-1")
        ReviewCount_element = soup.find('span', class_="stamped-summary-text")
        ProductName = P_name_element.text.strip()
        ReviewCount = ReviewCount_element.text.strip() if ReviewCount_element else 'Based on 0 Reviews'
        Rating = Rating_element.text.strip() if Rating_element else '0'
        variant = soup.find_all('tr')
        sub_pro = [span.text for row in variant for span in row.find_all('span')[:1] if row.find_all('span')]
        prices = []
        for row in variant:
            td_tags = row.find_all('td')
            for td in td_tags:
                if 'class' not in td.attrs and td.text.strip().startswith('$'):
                    prices.append(td.text.strip())
            cleaned_prices = [price.split('\n')[0].strip() if '\n' in price else price for price in prices]
        RegularPrice_element = soup.find_all('span', class_='original-price')
        RegularPrice= [RegularPrice_element[i].text.replace('Original Price:', '').strip() for i in range(len(RegularPrice_element))]
        SalePrice_element = soup.find_all('span', class_='sale-price')
        SalePrice= [SalePrice_element[i].text.replace('Sale Price:', '').strip() for i in range(len(SalePrice_element))]
        sub_pro = unique(sub_pro)
        if RegularPrice:
            for i in range(len(sub_pro)):
                row = {
                        'PageURL': URL,
                        'ProductName': ProductName,
                        'Season': Season,
                        'SubProduct' : sub_pro[i],
                        'RegularPrice': RegularPrice[i],
                        'SalePrice':SalePrice[i],
                        'ReviewCount': ReviewCount,
                        'OverAllRating': Rating
                    }
                df.append(row)
        else:
            for i in range(len(sub_pro)):
                row = {
                        'PageURL': URL,
                        'ProductName': ProductName,
                        'Season': Season,
                        'SubProduct' : sub_pro[i],
                        'RegularPrice': None,
                        'SalePrice':cleaned_prices[i],
                        'ReviewCount': ReviewCount,
                        'OverAllRating': Rating
                    }
                df.append(row)


    return df

In [40]:
Product_details=[]

In [46]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[0:], desc="GettingInfo", unit="URLs"):
    df = get_product_info(i)
    Product_details.extend(df)
daf = pd.DataFrame(Product_details)
daf.to_csv('EdenBrothers.csv',index=False)
driver.quit()    